# Contexto

Opendata AIG Brazil

A base de dados de ocorrências aeronáuticas é gerenciada pelo Centro de Investigação e Prevenção de Acidentes Aeronáuticos (CENIPA). Constam nesta base de dados as ocorrências aeronáuticas notificadas ao CENIPA nos últimos 10 anos e que ocorreram em solo brasileiro.

Dentre as informações disponíveis estão os dados sobre as aeronaves envolvidas, fatalidades, local, data, horário dos eventos e informações taxonômicas típicas das investigações de acidentes (AIG). São resguardadas a privacidade de pessoas físicas/jurídicas envolvidas conforme previsto pela Lei de Acesso à Informação (Lei n° 12.527, de 18 de novembro de 2011).

Essa base de dados é composta por informações preliminares provenientes do formulário CENIPA-05 (Ficha de Notificação de Ocorrências Aeronáuticas) e consolidada a partir dos relatórios de investigações publicados. Outra forma de visualização desses dados é pelo Painel SIPAER disponível na página do CENIPA.

Dados proveniente dos Programas de Gerenciamento de Prevenção administrados pelo CENIPA (Emissão de Raio Laser e Risco Baloeiro) não constam nesta base. Esses programas possuem formulários próprios para coleta de dados com foco exclusivo para gerenciamento do risco, enquanto os dados coletados pelo formulário CENIPA-05 têm foco principal na investigação de acidentes (AIG).
Tabelas (arquivos)

    OCORRÊNCIA.csv - Informações sobre as ocorrências.
    OCORRÊNCIA_TIPO.csv - Informações sobre o tipo de ocorrência.
    AERONAVE.csv - Informações sobre as aeronaves envolvidas nas ocorrências.
    FATOR_CONTRIBUINTE.csv - Informações sobre os fatores contribuinte das ocorrências que tiveram investigações finalizadas.
    RECOMENDAÇÃO.csv - Informações sobre as recomendações de segurança geradas nas ocorrências.
    Fonte: Sistema DÉDALO.

- Fonte 	http://www.fab.mil.br/cenipa/
- Autor 	Centro de Investigação e Prevenção de Acidentes Aeronáuticos
- Última Atualização 	5 de Outubro de 2021, 19:19 (UTC-03:00)
- Cobertura geográfica 	Brasil
- Cobertura temporal 	2010 a 2019
- Frequência de atualização 	Anual
- Granularidade geográfica 	Aeródromo
- Granularidade temporal 	Hora:Minuto

## Tabela Relacionamento

<img src="http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/modelo_dados.png" width="700 px">

# 0.0. IMPORTS

In [1]:
import pandas as pd
import numpy as np
import math

from IPython.display import display
pd.options.display.max_columns = None

## 0.1. Helper Functions

## 0.2. Loading data

In [7]:
ocorrencia = pd.read_csv('http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/ocorrencia.csv', sep=';')
tipo_ocorrencia = pd.read_csv('http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/ocorrencia_tipo.csv', sep=';')
aeronave = pd.read_csv('http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/aeronave.csv', sep=';')
fator_contribuinte = pd.read_csv('http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/fator_contribuinte.csv', sep=';')
rec_seguranca = pd.read_csv('http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/recomendacao.csv', sep=';')


In [19]:
#merge
aux1 = pd.merge( ocorrencia, tipo_ocorrencia, how='left', on='codigo_ocorrencia1' )
aux2 = pd.merge( aux1, aeronave, how='left', on='codigo_ocorrencia2' )
aux3 = pd.merge( aux2, fator_contribuinte, how='left', on='codigo_ocorrencia3' )
df_raw = pd.merge( aux3, rec_seguranca, how='left', on='codigo_ocorrencia4' )



In [27]:
df_raw.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17646 entries, 0 to 17645
Data columns (total 59 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   codigo_ocorrencia                17646 non-null  int64  
 1   codigo_ocorrencia1               17646 non-null  int64  
 2   codigo_ocorrencia2               17646 non-null  int64  
 3   codigo_ocorrencia3               17646 non-null  int64  
 4   codigo_ocorrencia4               17646 non-null  int64  
 5   ocorrencia_classificacao         17646 non-null  object 
 6   ocorrencia_latitude              16065 non-null  object 
 7   ocorrencia_longitude             16065 non-null  object 
 8   ocorrencia_cidade                17646 non-null  object 
 9   ocorrencia_uf                    17646 non-null  object 
 10  ocorrencia_pais                  17646 non-null  object 
 11  ocorrencia_aerodromo             17646 non-null  object 
 12  ocorrencia_dia    

<ipython-input-27-7128da69681d>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_raw.info(null_counts=True)


In [22]:
#df_raw.to_csv('./data/ocorrencias_aeronauticas_raw.csv', index=False)

In [2]:
df_raw = pd.read_csv('./data/ocorrencias_aeronauticas_raw.csv')

In [3]:
with pd.option_context('display.max_rows', 10):
    print(df_raw)

       codigo_ocorrencia  codigo_ocorrencia1  codigo_ocorrencia2  \
0                  40211               40211               40211   
1                  40349               40349               40349   
2                  40351               40351               40351   
3                  39527               39527               39527   
4                  39527               39527               39527   
...                  ...                 ...                 ...   
17641              80245               80245               80245   
17642              80247               80247               80247   
17643              80251               80251               80251   
17644              80257               80257               80257   
17645              80259               80259               80259   

       codigo_ocorrencia3  codigo_ocorrencia4 ocorrencia_classificacao  \
0                   40211               40211                INCIDENTE   
1                   40349          

# 1.0.  DESCRICAO DOS DADOS

In [4]:
df1 = df_raw.copy()

## 1.1. Rename Columns

## 1.2. Data Dimensions

In [89]:
print( 'Number of Rows: {}'.format( df1.shape[0] ) )
print( 'Number of Cols: {}'.format( df1.shape[1] ) )

Number of Rows: 17646
Number of Cols: 59


In [5]:
# check raw data main infos
df1.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17646 entries, 0 to 17645
Data columns (total 59 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   codigo_ocorrencia                17646 non-null  int64  
 1   codigo_ocorrencia1               17646 non-null  int64  
 2   codigo_ocorrencia2               17646 non-null  int64  
 3   codigo_ocorrencia3               17646 non-null  int64  
 4   codigo_ocorrencia4               17646 non-null  int64  
 5   ocorrencia_classificacao         17646 non-null  object 
 6   ocorrencia_latitude              16065 non-null  object 
 7   ocorrencia_longitude             16065 non-null  object 
 8   ocorrencia_cidade                17646 non-null  object 
 9   ocorrencia_uf                    17646 non-null  object 
 10  ocorrencia_pais                  17646 non-null  object 
 11  ocorrencia_aerodromo             17646 non-null  object 
 12  ocorrencia_dia    

<ipython-input-5-6769e3449d77>:2: FutureWarning: null_counts is deprecated. Use show_counts instead
  df1.info(null_counts=True)


In [ ]:
##Feature engineering
## Só sim e não
#divulgacao_relatorio_publicado -só tem sim e não  
#ocorrencia_saida_pista

# Filtragem 
#divulgacao_relatorio_numero       filtrar coluna 
# recomendacao_numero filtrar coluna  

## 1.3. Data Types

### Alterando os tipos dos dados para facilitar a manipulação de dados e reduzir o uso de recursos computacionais

In [79]:
df1.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia1                   int64
codigo_ocorrencia2                   int64
codigo_ocorrencia3                   int64
codigo_ocorrencia4                   int64
ocorrencia_classificacao            object
ocorrencia_latitude                 object
ocorrencia_longitude                object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_pais                     object
ocorrencia_aerodromo                object
ocorrencia_dia                      object
ocorrencia_hora                     object
investigacao_aeronave_liberada      object
investigacao_status                 object
divulgacao_relatorio_numero         object
divulgacao_relatorio_publicado      object
divulgacao_dia_publicacao           object
total_recomendacoes                  int64
total_aeronaves_envolvidas           int64
ocorrencia_saida_pista              object
ocorrencia_tipo                     object
ocorrencia_

In [6]:
#ocorrencia_dia - mudar para datetime 
df1.ocorrencia_dia = pd.to_datetime(df1['ocorrencia_dia'], format='%d/%m/%Y')
df1.ocorrencia_dia = pd.to_datetime(df1['ocorrencia_dia'], format='%Y-%m-%d')


# recomendacao_dia_encaminhamento - 0 se dia for data com má formato 
df1['recomendacao_dia_encaminhamento'] = df1['recomendacao_dia_encaminhamento'].apply( lambda x: 0 if ((x == '0002-11-29') or( x =='0002-11-24')or (x == '0002-11-17') or (x == '0002-11-27') or (x == '0002-11-26')) else x )
df1['recomendacao_dia_encaminhamento'] =   pd.to_datetime(df1['recomendacao_dia_encaminhamento'], format= '%Y-%m-%d')


#recomendacao_dia_feedback
df1['recomendacao_dia_feedback'] = df1['recomendacao_dia_feedback'].apply(lambda x: 0 if ((x == '0000-00-00') or (x == '0002-11-29') or (x == '0002-11-21') or (x == '0002-11-23')) else x)
df1['recomendacao_dia_feedback'] =   pd.to_datetime(df1['recomendacao_dia_feedback'], format= '%Y-%m-%d')

#recomendacao_dia_assinatura
df1['recomendacao_dia_assinatura'] =   pd.to_datetime(df1['recomendacao_dia_assinatura'], format= '%Y-%m-%d')

In [7]:
# convertendo os dados numéricos
for c in df1.select_dtypes( include=['int64'] ):
    df1[c] = df1[c].astype('int32')
    
# 36  aeronave_ano_fabricacao          17414 non-null  float64
df1['aeronave_ano_fabricacao'] = df1['aeronave_fatalidades_total'].astype('int32')

In [8]:
# 35  aeronave_assentos                17646 non-null  int32  
# aeronave_assentos - 0 para na - lidarei com os na mais tarde
df1['aeronave_assentos'].fillna(0, inplace=True )
df1.aeronave_assentos =df1['aeronave_assentos'].astype('int32')

In [9]:
# convertendo os dados categóricos
for c in df1.select_dtypes( include=['object'] ):
    df1[c] = df1[c].astype('category')

In [10]:
df1.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17646 entries, 0 to 17645
Data columns (total 59 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   codigo_ocorrencia                17646 non-null  int32         
 1   codigo_ocorrencia1               17646 non-null  int32         
 2   codigo_ocorrencia2               17646 non-null  int32         
 3   codigo_ocorrencia3               17646 non-null  int32         
 4   codigo_ocorrencia4               17646 non-null  int32         
 5   ocorrencia_classificacao         17646 non-null  category      
 6   ocorrencia_latitude              16065 non-null  category      
 7   ocorrencia_longitude             16065 non-null  category      
 8   ocorrencia_cidade                17646 non-null  category      
 9   ocorrencia_uf                    17646 non-null  category      
 10  ocorrencia_pais                  17646 non-null  category 

## 1.4. Check NA

In [11]:
df1.sample()

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,aeronave_pmd_categoria,aeronave_assentos,aeronave_ano_fabricacao,aeronave_pais_fabricante,aeronave_pais_registro,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total,fator_nome,fator_aspecto,fator_condicionante,fator_area,recomendacao_numero,recomendacao_dia_assinatura,recomendacao_dia_encaminhamento,recomendacao_dia_feedback,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_destinatario
16866,79383,79383,79383,79383,79383,INCIDENTE,-1.3801422,-48.476728,BELÉM,PA,BRASIL,SBBE,2020-06-08,19:03:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP,PTRMX,***,AVIÃO,EMBRAER,EMB-810C,PA34,PISTÃO,BIMOTOR,2073,2073,6,0,BRASIL,BRASIL,AVIÃO,TÁXI AÉREO,ORIXIMINÁ,VAL DE CANS / JÚLIO CEZAR RIBEIRO,CORRIDA APÓS POUSO,TÁXI AÉREO,NENHUM,0,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN


In [12]:
df1.isna().sum()

codigo_ocorrencia                      0
codigo_ocorrencia1                     0
codigo_ocorrencia2                     0
codigo_ocorrencia3                     0
codigo_ocorrencia4                     0
ocorrencia_classificacao               0
ocorrencia_latitude                 1581
ocorrencia_longitude                1581
ocorrencia_cidade                      0
ocorrencia_uf                          0
ocorrencia_pais                        0
ocorrencia_aerodromo                   0
ocorrencia_dia                         0
ocorrencia_hora                        1
investigacao_aeronave_liberada      1952
investigacao_status                  354
divulgacao_relatorio_numero          832
divulgacao_relatorio_publicado         0
divulgacao_dia_publicacao           4740
total_recomendacoes                    0
total_aeronaves_envolvidas             0
ocorrencia_saida_pista                 0
ocorrencia_tipo                        0
ocorrencia_tipo_categoria              0
taxonomia_tipo_i

## 1.5 Fill Out NA

In [ ]:
#recomendacao_numero                5698
#recomendacao_dia_assinatura        5698
#recomendacao_dia_encaminhamento    5698
#recomendacao_dia_feedback          9519
#recomendacao_conteudo              5699
#recomendacao_status                5698
#recomendacao_destinatario_sigla    5698
#recomendacao_destinatario          5698


In [84]:
#correncia_latitude  como todas com na tem 'ocorrencia_cidade', colocar 0 para manter as linhas
df1['ocorrencia_latitude'] = df1['ocorrencia_latitude'].cat.add_categories("0").fillna("0")

#ocorrencia_longitude                1581
df1['ocorrencia_longitude'] = df1['ocorrencia_longitude'].cat.add_categories("0").fillna("0")

#ocorrencia_hora       drop
df1.dropna(subset =['ocorrencia_hora'], inplace=True)

#investigacao_aeronave_liberada - Suposições:
## SIM se investigação status = FINALIZADO e recomendacao_status == 'CUMPRIDA' ou "CUMPRIDA DE FORMA ALTERNATIVA "
# NAO se divulgacao_relatorio_publicado == NAO 
# os na´s que sobreram drop =647 
df1.loc[ (df1['investigacao_aeronave_liberada'].isna()) & (df1['investigacao_status']== 'FINALIZADA') & (df1['recomendacao_status'] == ('CUMPRIDA' or 'CUMPRIDA DE FORMA ALTERNATIVA') ) , 'investigacao_aeronave_liberada'] = "SIM"
df1.loc[ ( df1['investigacao_aeronave_liberada'].isna() ) & ( df1.divulgacao_relatorio_publicado == 'NÃO' ) , 'investigacao_aeronave_liberada'] = 'NÃO'
df1.dropna(subset =['investigacao_aeronave_liberada'], inplace=True)


#investigacao_status                  354   
df1.dropna(subset =['investigacao_status'], inplace=True)

#divulgacao_relatorio_numero          832  filtrar coluna , não importante
df1['divulgacao_relatorio_numero'].fillna('***', inplace=True )

#divulgacao_dia_publicacao           4740  destes     4385 NÃO tem relatorio publicado , data - 0 
df1['divulgacao_dia_publicacao'] = df1['divulgacao_dia_publicacao'].cat.add_categories("0").fillna("0")

#aeronave_voo_origem                    1
df1.dropna(subset =['aeronave_voo_origem'], inplace=True)

# aeronave_motor_tipo
df1.dropna(subset =['aeronave_motor_tipo'], inplace=True)

## 4799  ou 28% do db como o obj. criei cat 'não informado' -para relatório ok                        
df1['fator_nome'] = df1['fator_nome'].cat.add_categories("não informado").fillna("não informado")                     
df1['fator_aspecto'] = df1['fator_aspecto'].cat.add_categories("não informado").fillna("não informado")
df1['fator_condicionante'] = df1['fator_condicionante'].cat.add_categories("não informado").fillna("não informado")
df1['fator_area'] = df1['fator_area'].cat.add_categories("não informado").fillna("não informado")

# recomendacao_dia_assinatura = quando publivcado = recomendacao_dia_assinatura == 'divulgacao_dia_publicacao'
df1.loc[ df1['recomendacao_dia_assinatura'].isna(), 'recomendacao_dia_assinatura' ] = df1.loc[ df1['recomendacao_dia_assinatura'].isna(), 'divulgacao_dia_publicacao' ]

# para ter esses valores, normalmente divulgacao_relatorio_publicado == SIM investigacao_status == finalizado
#'recomendacao_numero' + EXCLUIR COLUNA NA FILTRAGEM 
df1['recomendacao_numero'] = df1['recomendacao_numero'].cat.add_categories("NA - Investigacao NAO finalizada")
df1.loc[df1['recomendacao_numero'].isna()  & (df1['investigacao_status']== 'ATIVA') & (df1.divulgacao_relatorio_publicado == 'NÃO' ), 'recomendacao_numero'] = "NA - Investigacao NAO finalizada"


#recomendacao_dia_encaminhamento    5698
# não foi publicado, não tem como ter folow up = divulgacao_relatorio_publicado == "NÃO", então 0.
# suposicao= o Objetivo do relatorio não seja de folow up da resposta/retorno da empresa após divulgação, então vou colocar 0 nos na faltantes, por não considerar esse coluna importante = Filtrar coluna??
df1.loc[df1['recomendacao_dia_encaminhamento'].isna()  & (df1['divulgacao_relatorio_publicado']== 'NÃO') , 'recomendacao_dia_encaminhamento'] = 0
df1['recomendacao_dia_encaminhamento'].fillna(0, inplace=True )


ValueError: new categories must not include old categories: {'0'}

In [91]:
df1.loc[df1['recomendacao_numero'].isna() & (df1.divulgacao_relatorio_publicado == 'NÃO') ]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,aeronave_pmd_categoria,aeronave_assentos,aeronave_ano_fabricacao,aeronave_pais_fabricante,aeronave_pais_registro,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total,fator_nome,fator_aspecto,fator_condicionante,fator_area,recomendacao_numero,recomendacao_dia_assinatura,recomendacao_dia_encaminhamento,recomendacao_dia_feedback,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_destinatario
0,40211,40211,40211,40211,40211,INCIDENTE,***,***,RIO DE JANEIRO,RJ,BRASIL,****,2010-01-03,12:00:00,SIM,FINALIZADA,***,NÃO,0,0,1,NÃO,TRÁFEGO AÉREO,PERDA DE SEPARAÇÃO / COLISÃO EM VOO | TRÁFEGO ...,MAC,PRCHE,TÁXI AÉREO,HELICÓPTERO,SIKORSKY AIRCRAFT,S-76C,S76,TURBOEIXO,BIMOTOR,5307,5307,14,0,BRASIL,BRASIL,HELICÓPTERO,TÁXI AÉREO,FORA DE AERODROMO,FORA DE AERODROMO,DECOLAGEM,TÁXI AÉREO,NENHUM,0,não informado,não informado,não informado,não informado,NaN,0,0,NaT,NaN,NaN,NaN,NaN
1,40349,40349,40349,40349,40349,INCIDENTE,0,0,BELÉM,PA,BRASIL,SBBE,2010-01-03,11:05:00,SIM,FINALIZADA,***,NÃO,0,0,1,NÃO,COLISÃO COM AVE,COLISÃO COM AVE,BIRD,PRMAB,REGULAR,AVIÃO,AIRBUS INDUSTRIE,A320-232,A320,JATO,BIMOTOR,70000,70000,184,0,BRASIL,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,DECOLAGEM,REGULAR,LEVE,0,não informado,não informado,não informado,não informado,NaN,0,0,NaT,NaN,NaN,NaN,NaN
2,40351,40351,40351,40351,40351,INCIDENTE,0,0,RIO DE JANEIRO,RJ,BRASIL,SBRJ,2010-01-03,03:00:00,SIM,FINALIZADA,***,NÃO,0,0,1,NÃO,COLISÃO COM AVE,COLISÃO COM AVE,BIRD,PRMBW,REGULAR,AVIÃO,AIRBUS INDUSTRIE,A319-132,A319,JATO,BIMOTOR,75500,75500,154,0,BRASIL,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,INDETERMINADA,REGULAR,LEVE,0,não informado,não informado,não informado,não informado,NaN,0,0,NaT,NaN,NaN,NaN,NaN
7,40324,40324,40324,40324,40324,INCIDENTE,0,0,PELOTAS,RS,BRASIL,SBPK,2010-01-05,19:25:00,SIM,FINALIZADA,***,NÃO,0,0,1,NÃO,OUTROS,OUTROS,OTHR,PPPSB,REGULAR,AVIÃO,EMBRAER,EMB-120ER,E120,TURBOÉLICE,BIMOTOR,11990,11990,34,0,BRASIL,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,TÁXI,REGULAR,NENHUM,0,não informado,não informado,não informado,não informado,NaN,0,0,NaT,NaN,NaN,NaN,NaN
9,40215,40215,40215,40215,40215,INCIDENTE,***,***,COARI,AM,BRASIL,SBUY,2010-01-07,18:40:00,SIM,FINALIZADA,***,NÃO,0,0,1,SIM,TRÁFEGO AÉREO,PERDA DE SEPARAÇÃO / COLISÃO EM VOO | TRÁFEGO ...,MAC,PRTTH,REGULAR,AVIÃO,AEROSPATIALE AND ALENIA,ATR-42-500,AT45,TURBOÉLICE,BIMOTOR,18600,18600,52,0,BRASIL,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,POUSO,REGULAR,NENHUM,0,não informado,não informado,não informado,não informado,NaN,0,0,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17640,80244,80244,80244,80244,80244,INCIDENTE,-22.978841,-46.537076°,BRAGANÇA PAULISTA,SP,BRASIL,SBBP,2021-08-15,15:20:00,SIM,FINALIZADA,***,NÃO,0,0,1,NÃO,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP,PTMDN,***,AVIÃO,CESSNA AIRCRAFT,152,C152,PISTÃO,MONOMOTOR,757,757,2,0,BRASIL,BRASIL,AVIÃO,IN

In [89]:
df1.isna().sum()

codigo_ocorrencia                     0
codigo_ocorrencia1                    0
codigo_ocorrencia2                    0
codigo_ocorrencia3                    0
codigo_ocorrencia4                    0
ocorrencia_classificacao              0
ocorrencia_latitude                   0
ocorrencia_longitude                  0
ocorrencia_cidade                     0
ocorrencia_uf                         0
ocorrencia_pais                       0
ocorrencia_aerodromo                  0
ocorrencia_dia                        0
ocorrencia_hora                       0
investigacao_aeronave_liberada        0
investigacao_status                   0
divulgacao_relatorio_numero           0
divulgacao_relatorio_publicado        0
divulgacao_dia_publicacao             0
total_recomendacoes                   0
total_aeronaves_envolvidas            0
ocorrencia_saida_pista                0
ocorrencia_tipo                       0
ocorrencia_tipo_categoria             0
taxonomia_tipo_icao                   0


In [61]:
df2 = df1.copy()

In [16]:
df2.sample()

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,aeronave_pmd_categoria,aeronave_assentos,aeronave_ano_fabricacao,aeronave_pais_fabricante,aeronave_pais_registro,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total,fator_nome,fator_aspecto,fator_condicionante,fator_area,recomendacao_numero,recomendacao_dia_assinatura,recomendacao_dia_encaminhamento,recomendacao_dia_feedback,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_destinatario
3722,43814,43814,43814,43814,43814,ACIDENTE,-23.1866666667,-46.9436111111,JUNDIAÍ,SP,BRASIL,****,2011-02-04,20:30:00,SIM,FINALIZADA,A-502/CENIPA/2021,NÃO,0,0,1,NÃO,PANE SECA,COMBUSTÍVEL | PANE SECA,FUEL,PRRPE,***,AVIÃO,CIRRUS DESIGN,SR22,SR22,PISTÃO,MONOMOTOR,1542,1542,4,0,BRASIL,BRASIL,AVIÃO,PARTICULAR,AEROPORTO ESTADUAL DE JUNDIAÍ,SOROCABA,DECOLAGEM,PRIVADA,SUBSTANCIAL,0,não informado,não informado,não informado,não informado,NaN,0,0,NaT,NaN,NaN,NaN,NaN


In [ ]:
#recomendacao_numero                5176
#recomendacao_dia_assinatura           0
#recomendacao_dia_encaminhamento    1352
#recomendacao_dia_feedback          9519
#recomendacao_conteudo              5699
#recomendacao_status                5698
#recomendacao_destinatario_sigla    5698
#recomendacao_destinatario          5698

In [17]:
#df2['recomendacao_dia_encaminhamento'].isna().sum()
df2['recomendacao_dia_encaminhamento'].size

16643

In [18]:
135200/16643

8.123535420296822

In [62]:
df2.recomendacao_dia_encaminhamento.value_counts()

0                      4346
2012-02-22 00:00:00     684
2016-07-14 00:00:00     417
2013-08-12 00:00:00     362
2012-10-16 00:00:00     355
                       ... 
2016-11-01 00:00:00       1
2021-08-17 00:00:00       1
2014-10-08 00:00:00       1
2019-12-18 00:00:00       1
2020-10-22 00:00:00       1
Name: recomendacao_dia_encaminhamento, Length: 196, dtype: int64

In [23]:
df2['recomendacao_dia_encaminhamento'].isna().sum()

1352

In [80]:
aux = df2.loc[df2['recomendacao_dia_encaminhamento'].isna(), : ]
aux.recomendacao_dia_assinatura < '2016-01-01' 
aux.loc[aux.recomendacao_dia_assinatura < '2015-01-01' ]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,aeronave_pmd_categoria,aeronave_assentos,aeronave_ano_fabricacao,aeronave_pais_fabricante,aeronave_pais_registro,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total,fator_nome,fator_aspecto,fator_condicionante,fator_area,recomendacao_numero,recomendacao_dia_assinatura,recomendacao_dia_encaminhamento,recomendacao_dia_feedback,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_destinatario
259,39487,39487,39487,39487,39487,INCIDENTE GRAVE,-10.290000,-48.357778,PALMAS,TO,BRASIL,SBPJ,2010-01-26,16:50:00,SIM,FINALIZADA,IG-575/CENIPA/2014,SIM,2014-10-13,0,1,NÃO,POUSO SEM TREM,CONTATO ANORMAL COM A PISTA | POUSO SEM TREM,ARC,PRSAM,***,AVIÃO,BEECH AIRCRAFT,F33A,BE33,PISTÃO,MONOMOTOR,1588,1588,4,0,BRASIL,BRASIL,AVIÃO,PARTICULAR,SANTA GENOVEVA/GOIÂNIA,BRIGADEIRO LYSIAS RODRIGUES,POUSO,PRIVADA,SUBSTANCIAL,0,ESQUECIMENTO DO PILOTO,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL,NaN,2014-10-13,NaT,NaT,NaN,NaN,NaN,NaN
2582,43021,43021,43021,43021,43021,ACIDENTE,-28.7775,-55.8144444444,SÃO BORJA,RS,BRASIL,****,2010-10-01,11:30:00,SIM,FINALIZADA,A-592/CENIPA/2014,SIM,2014-10-01,0,1,NÃO,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP,PTUHJ,***,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-202,IPAN,PISTÃO,MONOMOTOR,1800,1800,1,0,BRASIL,BRASIL,AVIÃO,ESPECIALIZADA,SÃO BORJA,FORA DE AERODROMO,OUTRA FASE,ESPECIALIZADA,SUBSTANCIAL,0,MANUTENÇÃO DA AERONAVE,DESEMPENHO DO SER HUMANO,MANUTENÇÃO DA AERONAVE,FATOR OPERACIONAL,NaN,2014-10-01,NaT,NaT,NaN,NaN,NaN,NaN
3354,43724,43724,43724,43724,43724,ACIDENTE,-22.7097222222,-47.0827777778,COSMÓPOLIS,SP,BRASIL,****,2010-12-29,12:00:00,***,FINALIZADA,A-521/CENIPA/2014,SIM,2014-08-22,0,1,NÃO,COLISÃO COM OBSTÁCULOS NO SOLO,COLISÃO NO SOLO | COLISÃO COM OBSTÁCULOS NO SOLO,GCOL,PTUHQ,***,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-202,IPAN,PISTÃO,MONOMOTOR,1800,1800,1,0,BRASIL,BRASIL,AVIÃO,AGRÍCOLA,FORA DE AERODROMO,FORA DE AERODROMO,DECOLAGEM,AGRÍCOLA,SUBSTANCIAL,0,JULGAMENTO DE PILOTAGEM,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL,NaN,2014-08-22,NaT,NaT,NaN,NaN,NaN,NaN
3508,43703,43703,43703,43703,43703,ACIDENTE,-15.3102777778,-55.155,CAMPO VERDE,MT,BRASIL,****,2011-01-15,19:00:00,***,FINALIZADA,A-593/CENIPA/2014,SIM,2014-10-01,0,1,NÃO,PERDA DE CONTROLE NO SOLO,PERDA DE CONTROLE NO SOLO,LOC-G,PTURK,***,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-202,IPAN,PISTÃO,MONOMOTOR,1800,1800,1,0,BRASIL,BRASIL,AVIÃO,PARTICULAR,PARQUE ANA COSTA,FORA DE AERODROMO,POUSO,PRIVADA,SUBSTANCIAL,0,CONDIÇÕES METEOROLÓGICAS ADVERSAS,ELEMENTOS RELACIONADOS AO AMBIENTE OPERACIONAL,***,FATOR OPERACIONAL,NaN,2014-10-01,NaT,NaT,NaN,NaN,NaN,NaN
3509,43703,43703,43703,43703,43703,ACIDENTE,-15.3102777778,-55.155,CAMPO VERDE,MT,BRASIL,****,2011-01-15,19:00:00,***,FINALIZADA,A-593/CENIPA/2014,SIM,2014-10-01,0,1,NÃO,PERDA DE CONTROLE NO SOLO,PERDA DE CONTROLE NO SOLO,LOC-G,PTURK,***,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-202,IPAN,PISTÃO,MONOMOTOR,1800,1800,1,0,BRASIL,BRASIL,AVIÃO,PARTICULAR,PARQUE ANA COSTA,FORA DE AERODROMO,POUSO,PRIVADA,SUBSTANCIAL,0,JULGAMENTO DE PILOTAGEM,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPER

In [40]:
df2.loc[(df2['recomendacao_dia_encaminhamento'].isna() ) & (df2.recomendacao_dia_assinatura > '2017-01-01') ]

TypeError: '>' not supported between instances of 'Timestamp' and 'str'

In [37]:
df1.loc[ df1['investigacao_status']== 'FINALIZADA' , 'fator_aspecto'].value_counts()

DESEMPENHO DO SER HUMANO                          6247
ASPECTO PSICOLÓGICO                               4390
não informado                                     4278
ELEMENTOS RELACIONADOS AO AMBIENTE OPERACIONAL     290
INFRAESTRUTURA AEROPORTUÁRIA                       225
ASPECTO MÉDICO                                     217
OUTRO                                              217
ASPECTO DE FABRICAÇÃO                               67
***                                                 57
ASPECTO DE PROJETO                                  53
ERGONOMIA                                           36
INFRAESTRUTURA DE TRÁFEGO AÉREO                     33
ASPECTO DE MANUSEIO DO MATERIAL                      7
Name: fator_aspecto, dtype: int64

In [38]:
df1.loc[ df1['divulgacao_relatorio_publicado']== 'NÃO'  , 'investigacao_aeronave_liberada'].value_counts()

SIM    2504
***    1724
NÃO     160
Name: investigacao_aeronave_liberada, dtype: int64

In [260]:
df2.investigacao_aeronave_liberada.value_counts()

SIM    12444
***     5170
NÃO       31
Name: investigacao_aeronave_liberada, dtype: int64